In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import datetime as dt

In [23]:
df = pd.read_csv('Illinois_2021.csv', low_memory=False)

In [ ]:
##################################################################################################

In [ ]:
###### MODEL1: OUTPUT = APPLICATION STATUS ######

In [ ]:
##### CLEANING #####

In [25]:
# DROPPING COLUMNS WITH NO VALUE
df.drop(['activity_year','state_code','census_tract','county_code','loan_type','derived_ethnicity',
         'derived_race','derived_sex', 'denial_reason-1','denial_reason-2','denial_reason-3','denial_reason-4',
         'applicant_age_above_62','co-applicant_age_above_62','tract_to_msa_income_percentage'], axis = 1, inplace = True)

In [26]:
# Filter the DF so that only accept or reject outcomes are present
df = df[df['action_taken'].isin([1,2,3])]

In [27]:
# Turning the Application outcome (action_taken) into dummy variables
df['action_taken'] = np.where((df['action_taken']==1)|(df['action_taken']==2),1,0)

In [38]:
# Filtering for only Home Purchase Loans
df = df[df['loan_purpose']==1]

In [43]:
# Filtering for only mortgage applications and NOT covered loans
df = df[df['submission_of_application']!=3]
df = df[df['initially_payable_to_institution']!=3]

In [44]:
# Dropping columns that are only applicable to succesful applications
df.drop(['interest_rate','rate_spread','purchaser_type','initially_payable_to_institution'], axis = 1, inplace = True)

In [ ]:
#### EXPLORING PROPERTY AND AREA RELATED FEATURES WITH SUCCESS RATE ###

In [ ]:
### MSA CODE: 5 digit metropolitan statistical area or metropolitan division code: min 50K population

In [81]:
# MSA Code Popularity + Success Rate , (Count = Success count of the MSA)
msa = pd.DataFrame(df.groupby('derived_msa-md')['action_taken'].value_counts())
msa.columns = ['Count']
msa = msa.reset_index()
msa['%'] = 100 * msa['Count'] / msa.groupby('derived_msa-md')['Count'].transform('sum')
msa = msa.sort_values(['derived_msa-md', 'action_taken'], ascending = False)
msa = msa[msa['action_taken']==1].set_index(['derived_msa-md']).drop('action_taken',axis = 1)
msa.sort_values(by = "Count", ascending = False)

,Count,%
derived_msa-md,,
16984,114186,91.603090
99999,15252,88.633194
20994,14637,93.610898
29404,13288,93.268758
41180,12384,91.910346
37900,7671,94.575268
40420,5378,91.665246
44100,4077,93.337912
19340,3641,92.340857


In [103]:
# MSA Popularity + Success Rate, ex-Outlier code 16984 (Chicago, Naperville, Evanston)
msa1 = msa.loc[[99999, 44100, 41180, 40420, 37900, 29404, 28100, 20994, 19500,
            19340, 19180, 16580, 16060, 16020, 14010,0]]
# Linear Relationship... Not Significant between Popularity of MSA Code and % Success
print("Without Outlier:",st.linregress(x = msa1['Count'], y = msa1['%']))
print("\n\nWith Outlier:",st.linregress(x = msa['Count'], y = msa['%']))

Without Outlier: LinregressResult(slope=0.0015370667416556817, intercept=75.61146270231731, rvalue=0.42473292945597957, pvalue=0.10103545706858483, stderr=0.000875617158157988, intercept_stderr=6.658221838539992)


With Outlier: LinregressResult(slope=0.0001230049876445171, intercept=83.13912103160108, rvalue=0.17547451528379068, pvalue=0.5005350960977265, stderr=0.00017818522568511955, intercept_stderr=5.106762693872425)


In [154]:
# correlation between each MSA code and outcome: case by case
msa2 = pd.get_dummies(df[['action_taken','derived_msa-md']], columns = ['derived_msa-md'])
msacorr = msa2.corr()['action_taken'].sort_values(ascending = False).reset_index()
msacorr.columns = ['MSA','Corr']

def calculate_pvalues(msa2):
    msa2 = msa2.dropna()._get_numeric_data()
    msacols = pd.DataFrame(columns=msa2.columns)
    pvalues = msacols.transpose().join(msacols, how='outer')
    for r in msa2.columns:
        for c in msa2.columns:
            pvalues[r][c] = round(st.pearsonr(msa2[r], msa2[c])[1], 4)
    return pvalues
msapval = calculate_pvalues(msa2)['action_taken'].sort_values(ascending = False).reset_index()
msapval.columns = ['MSA','PVal']


msacorr.merge(msapval)[1:]

,MSA,Corr,PVal
1,derived_msa-md_20994,0.023046,0.0
2,derived_msa-md_37900,0.022934,0.0
3,derived_msa-md_29404,0.018762,0.0
4,derived_msa-md_16984,0.014508,0.0
5,derived_msa-md_44100,0.010497,0.0
6,derived_msa-md_16580,0.007396,0.0005
7,derived_msa-md_41180,0.006023,0.0045
8,derived_msa-md_19340,0.005230,0.0135
9,derived_msa-md_40420,0.002479,0.242
10,derived_msa-md_14010,-0.001349,0.5243


In [ ]:
# Comparing Not 16580 vs 16580 Success Rates


In [156]:
### Occupancy Type: 1 is principal residence, 2 is secondary, 3 is investment

In [158]:
# distribution of occupancy type applications
df['occupancy_type'].value_counts()

1    203233
3     16370
2      3227
Name: occupancy_type, dtype: int64

In [159]:
# Occupancy Type + Success Rate , (Count = Success count of the Occupancy Type)
ot = pd.DataFrame(df.groupby('occupancy_type')['action_taken'].value_counts())
ot.columns = ['Count']
ot = ot.reset_index()
ot['%'] = 100 * ot['Count'] / ot.groupby('occupancy_type')['Count'].transform('sum')
ot = ot.sort_values(['occupancy_type', 'action_taken'], ascending = False)
ot = ot[ot['action_taken']==1].set_index(['occupancy_type']).drop('action_taken',axis = 1)
ot.sort_values(by = "Count", ascending = False)

,Count,%
occupancy_type,,
1,185331,91.191391
3,15036,91.850947
2,2941,91.137279


In [161]:
# Occupancy Type + Success Rate
st.linregress(x = ot['Count'], y = ot['%'])

LinregressResult(slope=-1.5031814740874966e-06, intercept=91.49507534723232, rvalue=-0.385844971807126, pvalue=0.747819920385155, stderr=3.5941392625817837e-06, intercept_stderr=0.38588800161557957)

In [ ]:
# Comparing Occupancy Type 1-2-3  Success Rates